# Exemplo de Utilização da Ferramenta AutoGen


## Importar os Pacotes Necessários


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Criar o Cliente

Neste exemplo, vamos utilizar os [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para aceder ao LLM.

O `model` está definido como `gpt-4o-mini`. Experimente alterar o modelo para outro disponível no marketplace do GitHub Models para observar resultados diferentes.

Como um teste rápido, vamos executar um prompt simples - `Qual é a capital de França`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definir as Funções

Neste exemplo, vamos dar ao agente acesso a uma ferramenta que é uma função com uma lista de destinos de férias disponíveis e a respetiva disponibilidade.

Pode imaginar que este seria um cenário em que um agente de viagens poderia ter acesso, por exemplo, a uma base de dados de viagens.

À medida que explora este exemplo, sinta-se à vontade para tentar definir novas funções e ferramentas que o agente possa utilizar.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definir a Ferramenta de Função  
Para que o agente utilize `vacation_destinations` como um `FunctionTool`, precisamos defini-lo como tal.  

Também iremos fornecer uma descrição da ferramenta, o que ajuda o agente a identificar para que serve essa ferramenta em relação à tarefa solicitada pelo utilizador.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definir o agente

Agora podemos criar o agente no código abaixo. Definimos a `system_message` para dar instruções ao agente sobre como ajudar os utilizadores a encontrar destinos de férias.

Também configuramos o parâmetro `reflect_on_tool_use` como verdadeiro. Isto permite que o LLM utilize a resposta da chamada da ferramenta e envie a resposta em linguagem natural.

Pode definir o parâmetro como falso para ver a diferença.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Executar o Agente

Agora podemos executar o agente com a mensagem inicial do utilizador a pedir para fazer uma viagem a Tóquio.

Pode alterar este destino para ver como o agente responde à disponibilidade da cidade.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
